In [1]:
import yaml
import os

from gen_answer import AnswerGenerator
from gen_judgment import Judge
from show_result import Result

In [2]:
LLM_JUDGE = None                # judge model from API                      ("gpt-4-1106-preview")
LLM_MODEL = None                # testing model from API                    ("gemma-2-9b-it")

GEN_ANSWER_CONFIG_NAME = None   # necessary if setup is not default         ("temp-0.2_tokens-4k")
OVERRIDE_GEN_PARAMS = None      # optional if such setup already created    ({temperature: 0.2, max_tokens: 4096})

JUDGE_CONFIG_NAME = None        # necessary if setup is not default         ("without_pairwise")
OVERRIDE_JUDGE_PARAMS = None    # optional if such setup already created    ({pairwise: False})

In [3]:
default_gen_config_path = "default_config/gen_answer_config.yaml"
default_judge_config_path = "default_config/judge_config.yaml"
api_config_path = "default_config/api_config.yaml"
data_path = "data"


with open(default_gen_config_path, 'r') as f:
    gen_config = yaml.safe_load(f)

with open(default_judge_config_path, 'r') as f:
    judge_config = yaml.safe_load(f)

with open(api_config_path, 'r') as f:
    api_config = yaml.safe_load(f)

## Generation

In [4]:
def equal_configs(dict1, dict2):
    dict1_filtered = {k: v for k, v in dict1.items() if k != "model_list"}
    dict2_filtered = {k: v for k, v in dict2.items() if k != "model_list"}

    return dict1_filtered == dict2_filtered


def config_name_provided(new_config, default_config_path, name):
    with open(default_config_path, 'r') as f:
        default_config = yaml.safe_load(f)
        if not equal_configs(new_config, default_config) and not name:
            return False    
            
    return True


def update_config(new_config, config_file, new_model):
    if os.path.isfile(config_file):
        with open(config_file, 'r') as f:
            config = yaml.safe_load(f)
        assert equal_configs(config, new_config)

        if new_model and (new_model not in config["model_list"]):
            config["model_list"].append(new_model)
    else:
        os.makedirs(os.path.dirname(config_file), exist_ok=True)
        with open(config_file, "w") as f:
            yaml.dump(new_config, f, default_flow_style=False, sort_keys=False)




if LLM_MODEL:
    assert isinstance(LLM_MODEL, str)
    assert LLM_MODEL in api_config
assert config_name_provided(new_config=gen_config, default_config_path=default_gen_config_path, name=GEN_ANSWER_CONFIG_NAME)
gen_config_name = (GEN_ANSWER_CONFIG_NAME if GEN_ANSWER_CONFIG_NAME else "default")


if OVERRIDE_GEN_PARAMS:
    gen_config.update(OVERRIDE_GEN_PARAMS)

if LLM_MODEL:
    gen_config["model_list"].append(LLM_MODEL)

gen_config_file = os.path.join(data_path, gen_config["bench_name"], "model_answer", gen_config_name, "config", "gen_answer_config.yaml")

update_config(new_config=gen_config, config_file=gen_config_file, new_model=LLM_MODEL)

In [5]:
generation_path = os.path.join(data_path, gen_config["bench_name"], "model_answer", gen_config_name)
generator = AnswerGenerator(generation_path, api_config_path)
print(generator)
generator.generate()


{'name': 'config of answer generation for arena-hard-v0.1', 'bench_name': 'arena-hard-v0.1', 'temperature': 0.0, 'max_tokens': 2048, 'num_choices': 1, 'model_list': ['gpt-3.5-turbo-0125', 'gpt-4-1106-preview', 'gpt-4o-mini', 'gpt-3.5-turbo-1106']}

Output to data/arena-hard-v0.1/model_answer/default/gpt-3.5-turbo-0125.jsonl
500 number of existing answers


0it [00:00, ?it/s]


Output to data/arena-hard-v0.1/model_answer/default/gpt-4-1106-preview.jsonl
500 number of existing answers


0it [00:00, ?it/s]


Output to data/arena-hard-v0.1/model_answer/default/gpt-4o-mini.jsonl
500 number of existing answers


0it [00:00, ?it/s]


Output to data/arena-hard-v0.1/model_answer/default/gpt-3.5-turbo-1106.jsonl
500 number of existing answers


0it [00:00, ?it/s]


## Judgment

In [6]:
if LLM_JUDGE:
    assert isinstance(LLM_JUDGE, str)
    assert LLM_JUDGE in api_config
assert config_name_provided(new_config=judge_config, default_config_path=default_judge_config_path, name=JUDGE_CONFIG_NAME)
assert "judge_model" in judge_config

if OVERRIDE_JUDGE_PARAMS:
    judge_config.update(OVERRIDE_JUDGE_PARAMS)

if LLM_JUDGE:
    judge_config["judge_model"] = LLM_JUDGE

judge_config_name = judge_config["judge_model"] + '_' + (JUDGE_CONFIG_NAME if JUDGE_CONFIG_NAME else "default")

if LLM_MODEL:
    judge_config["model_list"].append(LLM_MODEL)

judge_config_file = os.path.join(data_path, judge_config["bench_name"], "model_judgment", judge_config_name, "config", "judge_config.yaml")

update_config(new_config=judge_config, config_file=judge_config_file, new_model=LLM_MODEL)

In [7]:
judgment_path = os.path.join(data_path, judge_config["bench_name"], "model_judgment", judge_config_name)
judge = Judge(judgment_path, generation_path, api_config_path)
print(judge)
judge.judge()


judge model: gpt-4-1106-preview, baseline: True, baseline model: gpt-3.5-turbo-0125,reference: False, reference models: None, temperature: 0, max tokens: 4096, pairwise: True

500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgments
500 number of existing judgment

0it [00:00, ?it/s]


## Result

In [10]:
result = Result(generation_path, judgment_path)
print(result)
result.show()


{'bench_name': 'arena-hard-v0.1', 'judge_name': 'gpt-4-1106-preview', 'baseline': 'gpt-3.5-turbo-0125', 'load_battles': False, 'load_bootstrap': False, 'show_elo': False, 'length_control': False, 'weight': 3, 'num_rounds': 100, 'output': False, 'first_game_only': False}

Turning judgment results into battles...


bootstrap: 100%|██████████| 100/100 [00:06<00:00, 16.50it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Model                                    ┃ Score ┃      95% CI ┃ Avg. #Tokens ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ gpt-4-1106-preview                       │  90.9 │ (-1.0, 1.3) │          541 │
│ gpt-4o-mini                              │  83.9 │ (-1.9, 1.5) │          448 │
│ T-Tech-T-pro-it-1.0                      │  83.8 │ (-1.7, 1.5) │          502 │
│ gigachat_max_26.20_uncen                 │  82.7 │ (-1.4, 1.7) │          514 │
│ gigachat_max_with_censor                 │  80.0 │ (-1.8, 1.7) │          515 │
│ vikhr-nemo-12b-instruct-r-21-09-24       │  79.8 │ (-1.7, 1.8) │          627 │
│ gemma-2-9b-it-sppo-iter3                 │  73.6 │ (-1.8, 1.7) │          509 │
│ T-Tech-T-lite-it-1.0                     │  71.0 │ (-2.2, 1.9) │          544 │
│ qwen2.5-14b-instruct                     │  70.5 │ (-2.5, 2.2) │          434 │
│ gigachat_pro_26.20_uncen                 │  70.4 │ (-1.6, 2.0) │          549 │
│ gemma-2-9b-it                            │  69.2 │ (-2.1, 1.9) │          459 │
│ CohereForAI/aya-expanse-8b               │  67.1 │ (-1.7, 1.9) │          698 │
│ t-lite-instruct-0.1                      │  64.7 │ (-2.6, 2.6) │          810 │
│ vikhr-llama3.1-8b-instruct-r-21-09-24    │  63.4 │ (-2.0, 2.3) │          618 │
│ suzume-llama-3-8B-multilingual-orpo-bor… │  57.1 │ (-2.2, 1.6) │          682 │
│ gigachat_lite_26.20_uncen                │  56.4 │ (-2.2, 2.5) │          561 │
│ phi-3-medium-4k-instruct                 │  55.1 │ (-2.1, 1.9) │          566 │
│ mistral-nemo-instruct-2407               │  50.5 │ (-2.8, 2.4) │          403 │
│ yandex_gpt_pro_v4_26102024               │  50.5 │ (-2.5, 2.0) │          384 │
│ sfr-iterative-dpo-llama-3-8b-r           │  50.1 │ (-2.4, 2.0) │          516 │
│ gpt-3.5-turbo-0125                       │  50.0 │  (0.0, 0.0) │          220 │
│ glm-4-9b-chat                            │  49.8 │ (-2.1, 2.3) │          568 │
│ c4ai-command-r-v01                       │  49.0 │ (-2.6, 1.9) │          529 │
│ llama-3-instruct-8b-sppo-iter3           │  47.5 │ (-2.4, 2.0) │          502 │
│ suzume-llama-3-8b-multilingual           │  45.7 │ (-2.0, 2.2) │          641 │
│ yandex_gpt_pro                           │  45.1 │ (-2.5, 1.9) │          345 │
│ hermes-2-theta-llama-3-8b                │  44.1 │ (-2.0, 2.3) │          485 │
│ yandex_gpt_lite_v4_26102024              │  42.7 │ (-2.0, 2.2) │          328 │
│ gpt-3.5-turbo-1106                       │  41.5 │ (-1.6, 2.4) │          191 │
│ llama-3-smaug-8b                         │  40.8 │ (-2.2, 2.0) │          524 │
│ llama-3-8b-saiga-suzume-ties             │  39.9 │ (-2.2, 2.7) │          763 │
│ starling-lm-7b-beta                      │  39.8 │ (-2.3, 2.4) │          629 │
│ vikhr-it-5.4-fp16-orpo-v2                │  39.3 │ (-1.7, 2.1) │          379 │
│ saiga_llama3_8b_v6                       │  39.2 │ (-2.2, 2.3) │          471 │
│ llama-3-instruct-8b-simpo                │  38.0 │ (-2.2, 1.8) │          417 │
│ qwen2-7b-instruct                        │  37.5 │ (-2.3, 2.2) │          340 │
│ paralex-llama-3-8b-sft                   │  37.4 │ (-2.1, 2.4) │          688 │
│ aya-23-8b                                │  36.3 │ (-2.1, 2.2) │          554 │
│ meta-llama-3-8b-instruct                 │  35.1 │ (-2.2, 1.8) │          450 │
│ openchat-3.5-0106                        │  33.8 │ (-2.1, 1.8) │          492 │
│ mistral-7b-instruct-v0.3                 │  32.9 │ (-1.7, 1.9) │          469 │
│ vikhr-it-5.2-fp16-cp                     │  31.7 │ (-1.5, 1.9) │          543 │
│ hermes-2-pro-llama-3-8b                  │  30.8 │ (-2.4, 2.2) │          463 │
│ openchat-3.6-8b-20240522                 │  30.3 │ (-2.5, 2.0) │          428 │
│ vikhr-it-5.3-fp16-32k                    │  27.8 │ (-1.9, 2.2) │          519 │
│ vikhr-it-5.3-fp16                        │  22.7 │ (-1.6, 1.7)